In [2]:
import cudf, cuml, cupy
print('RAPIDS',cuml.__version__)

RAPIDS 21.10.02


In [3]:
# LOAD TRAIN UNTO THE GPU WITH CUDF
train_gf = cudf.read_csv('/kaggle/input/daphnet-dataset/data0.csv')
print('train shape is', train_gf.shape)
train_gf.head()

train shape is (303047, 769)


,input0,input1,input2,input3,input4,input5,input6,input7,input8,input9,...,input759,input760,input761,input762,input763,input764,input765,input766,input767,label
0,-45.0,-18.0,18.0,36.0,36.0,27.0,27.0,-18.0,-90.0,-90.0,...,181.0,202.0,202.0,171.0,202.0,212.0,202.0,202.0,191.0,0.0
1,-18.0,18.0,36.0,36.0,27.0,27.0,-18.0,-90.0,-90.0,-136.0,...,202.0,202.0,171.0,202.0,212.0,202.0,202.0,191.0,181.0,0.0
2,18.0,36.0,36.0,27.0,27.0,-18.0,-90.0,-90.0,-136.0,-145.0,...,202.0,171.0,202.0,212.0,202.0,202.0,191.0,181.0,222.0,0.0
3,36.0,36.0,27.0,27.0,-18.0,-90.0,-90.0,-136.0,-145.0,-163.0,...,171.0,202.0,212.0,202.0,202.0,191.0,181.0,222.0,212.0,0.0
4,36.0,27.0,27.0,-18.0,-90.0,-90.0,-136.0,-145.0,-163.0,-236.0,...,202.0,212.0,202.0,202.0,191.0,181.0,222.0,212.0,202.0,0.0


In [4]:
X = train_gf.iloc[:, :-1]
X

,input0,input1,input2,input3,input4,input5,input6,input7,input8,input9,...,input758,input759,input760,input761,input762,input763,input764,input765,input766,input767
0,-45.0,-18.0,18.0,36.0,36.0,27.0,27.0,-18.0,-90.0,-90.0,...,161.0,181.0,202.0,202.0,171.0,202.0,212.0,202.0,202.0,191.0
1,-18.0,18.0,36.0,36.0,27.0,27.0,-18.0,-90.0,-90.0,-136.0,...,181.0,202.0,202.0,171.0,202.0,212.0,202.0,202.0,191.0,181.0
2,18.0,36.0,36.0,27.0,27.0,-18.0,-90.0,-90.0,-136.0,-145.0,...,202.0,202.0,171.0,202.0,212.0,202.0,202.0,191.0,181.0,222.0
3,36.0,36.0,27.0,27.0,-18.0,-90.0,-90.0,-136.0,-145.0,-163.0,...,202.0,171.0,202.0,212.0,202.0,202.0,191.0,181.0,222.0,212.0
4,36.0,27.0,27.0,-18.0,-90.0,-90.0,-136.0,-145.0,-163.0,-236.0,...,171.0,202.0,212.0,202.0,202.0,191.0,181.0,222.0,212.0,202.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303042,-927.0,-918.0,-909.0,-918.0,-918.0,-909.0,-909.0,-918.0,-900.0,-909.0,...,212.0,222.0,232.0,191.0,202.0,242.0,242.0,202.0,212.0,232.0
303043,-918.0,-909.0,-918.0,-918.0,-909.0,-909.0,-918.0,-900.0,-909.0,-918.0,...,222.0,232.0,191.0,202.0,242.0,242.0,202.0,212.0,232.0,191.0
303044,-909.0,-918.0,-918.0,-909.0,-909.0,-918.0,-900.0,-909.0,-918.0,-909.0,...,232.0,191.0,202.0,242.0,242.0,202.0,212.0,232.0,191.0,212.0
303045,-918.0,-918.0,-909.0,-909.0,-918.0,-900.0,-909.0,-918.0,-909.0,-890.0,...,191.0,202.0,242.0,242.0,202.0,212.0,232.0,191.0,212.0,202.0


In [5]:
y = train_gf.label.astype(int)
y

0         0
1         0
2         0
3         0
4         0
         ..
303042    0
303043    0
303044    0
303045    0
303046    0
Name: label, Length: 303047, dtype: int64

In [6]:
X_train, X_test, y_train, y_test = cuml.train_test_split(X, y, test_size=0.2, random_state=123)

In [7]:
def train(model):
    model.fit(X_train, y_train)
    h_test = model.predict(X_test)
    accuracy = cuml.metrics.accuracy.accuracy_score(y_test, h_test)
    mat = cuml.metrics.confusion_matrix(y_test, h_test)
    tn, fp, fn, tp = mat.ravel()
    roc_auc = cuml.metrics.roc_auc_score(y_test, h_test)
    sensitivity = recall = tp / (tp+fn)
    specificity = recall = tn / (tn+fp)
    precision = tp / (tp + fp)
    del h_test
    return dict(
        accuracy = accuracy,
        tn = tn,
        fp = fp,
        fn = fn,
        tp = tp,
        roc_auc = roc_auc,
        sensitivity = sensitivity,
        specificity = specificity,
        precision = precision
    )

In [8]:
lr = cuml.linear_model.LogisticRegression()
cudf.DataFrame(train(lr))

,accuracy,tn,fp,fn,tp,roc_auc,sensitivity,specificity,precision
0,0.879803,53321,377,6908,3,0.496707,0.000434,0.992979,0.007895


In [9]:
svm = cuml.svm.SVC()
cudf.DataFrame(train(svm))

,accuracy,tn,fp,fn,tp,roc_auc,sensitivity,specificity,precision
0,0.912109,53258,440,4887,2024,0.642336,0.292866,0.991806,0.821429


In [10]:
knn = cuml.neighbors.KNeighborsClassifier(n_neighbors=55)
cudf.DataFrame(train(knn))

,accuracy,tn,fp,fn,tp,roc_auc,sensitivity,specificity,precision
0,0.886716,53155,543,6323,588,0.537485,0.085082,0.989888,0.519894


In [11]:
rf = cuml.ensemble.RandomForestClassifier()
cudf.DataFrame(train(rf))

/opt/conda/lib/python3.7/site-packages/cuml/internals/api_decorators.py:567: UserWarning: To use pickling or GPU-based prediction first train using float32 data to fit the estimator
  ret_val = func(*args, **kwargs)
/opt/conda/lib/python3.7/site-packages/cuml/internals/api_decorators.py:586: UserWarning: GPU based predict only accepts np.float32 data. The model was trained on np.float64 data hence cannot use GPU-based prediction! 
Defaulting to CPU-based Prediction. 
To predict on float-64 data, set parameter predict_model = 'CPU'
  ret_val = func(*args, **kwargs)


,accuracy,tn,fp,fn,tp,roc_auc,sensitivity,specificity,precision
0,0.900609,53579,119,5905,1006,0.571674,0.145565,0.997784,0.894222


In [12]:
# checkpoint_path = 'models/rf0.tl'
# # Export cuML RF model as Treelite checkpoint
# rf.convert_to_treelite_model().to_treelite_checkpoint(checkpoint_path)